In [36]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier

In [6]:
train = pd.read_csv('../Train/train_set_geo_levels_sum_mult.csv')

train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,geo_level_1_id_sum,geo_level_2_id_sum,geo_level_3_id_sum,geo_level_multiply,geo_level_sum,damage_grade
0,6,487,12198,2,30,6,5,1,1,0,...,0,0,0,1,24381,270,37,35642556,12691,3
1,8,900,2812,2,10,8,7,0,1,0,...,0,0,0,1,19080,199,16,20246400,3720,2
2,21,363,8973,2,10,5,5,0,1,0,...,0,0,0,1,14889,1760,136,68401179,9357,3
3,22,418,10694,2,10,6,5,0,1,0,...,0,0,0,1,6252,205,31,98342024,11134,2
4,11,131,1488,3,30,8,9,1,0,0,...,0,0,0,1,8220,1038,122,2144208,1630,3


In [7]:
X, y = train.iloc[:,:-1], train.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

## Modelo 1

In [8]:
estimators = []

estimators.append(("rf", RandomForestClassifier(class_weight='balanced', min_samples_leaf=4,
                       min_samples_split=5, n_estimators=70, random_state=123)))
estimators.append(("lgbm", lgb.LGBMClassifier(force_col_wise=False, force_row_wise=False, learning_rate=0.5,
               max_depth=10, n_estimators=40, num_leaves=30, objective='multiclass', random_state=123)))
estimators.append(("cat", CatBoostClassifier(silent=True)))
estimators.append(("xgb", xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.4, max_delta_step=0, max_depth=14,
              min_child_weight=1, missing=np.nan, monotone_constraints='()',
              n_estimators=11, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=123, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)))

In [25]:
model = VotingClassifier(estimators=estimators, voting='soft')

In [26]:
model.fit(X_train, y_train)

/home/ben/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:27:49] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(class_weight='balanced',
                                                     min_samples_leaf=4,
                                                     min_samples_split=5,
                                                     n_estimators=70,
                                                     random_state=123)),
                             ('lgbm',
                              LGBMClassifier(force_col_wise=False,
                                             force_row_wise=False,
                                             learning_rate=0.5, max_depth=10,
                                             n_estimators=40, num_leaves=30,
                                             objective='multiclass',
                                             random_state=123)),
                             ('cat',
                              <catboost.core.CatB...
                                            i

In [27]:
preds = model.predict(X_test)
f1 = f1_score(y_test, preds, average='micro')
print("F1 Score", f1)

F1 Score 0.7420283181765857


## Modelo 2

In [30]:
estimators_2 = []

estimators_2.append(("rf", RandomForestClassifier()))
estimators_2.append(("cat", CatBoostClassifier()))
estimators_2.append(("lgb", lgb.LGBMClassifier()))
estimators_2.append(("xgb", xgb.XGBClassifier()))

In [31]:
model_2 = VotingClassifier(estimators=estimators_2, voting='soft')

In [33]:
model_2.fit(X_train, y_train)

Learning rate set to 0.104521
0:	learn: 1.0402484	total: 49ms	remaining: 48.9s
1:	learn: 0.9934653	total: 100ms	remaining: 49.9s
2:	learn: 0.9561273	total: 151ms	remaining: 50.2s
3:	learn: 0.9257064	total: 204ms	remaining: 50.8s
4:	learn: 0.9006279	total: 257ms	remaining: 51.2s
5:	learn: 0.8779818	total: 307ms	remaining: 50.9s
6:	learn: 0.8589039	total: 356ms	remaining: 50.4s
7:	learn: 0.8426633	total: 402ms	remaining: 49.8s
8:	learn: 0.8284520	total: 453ms	remaining: 49.9s
9:	learn: 0.8151112	total: 500ms	remaining: 49.5s
10:	learn: 0.8041366	total: 549ms	remaining: 49.4s
11:	learn: 0.7942585	total: 594ms	remaining: 48.9s
12:	learn: 0.7855803	total: 647ms	remaining: 49.1s
13:	learn: 0.7773544	total: 704ms	remaining: 49.6s
14:	learn: 0.7708814	total: 755ms	remaining: 49.6s
15:	learn: 0.7650231	total: 805ms	remaining: 49.5s
16:	learn: 0.7598097	total: 853ms	remaining: 49.3s
17:	learn: 0.7556139	total: 905ms	remaining: 49.4s
18:	learn: 0.7509031	total: 962ms	remaining: 49.7s
19:	learn: 0

163:	learn: 0.6611737	total: 8.47s	remaining: 43.2s
164:	learn: 0.6609777	total: 8.53s	remaining: 43.2s
165:	learn: 0.6606097	total: 8.59s	remaining: 43.1s
166:	learn: 0.6602702	total: 8.65s	remaining: 43.2s
167:	learn: 0.6600845	total: 8.71s	remaining: 43.1s
168:	learn: 0.6599526	total: 8.76s	remaining: 43.1s
169:	learn: 0.6595366	total: 8.81s	remaining: 43s
170:	learn: 0.6591825	total: 8.86s	remaining: 43s
171:	learn: 0.6589736	total: 8.91s	remaining: 42.9s
172:	learn: 0.6586116	total: 8.97s	remaining: 42.9s
173:	learn: 0.6584378	total: 9.02s	remaining: 42.8s
174:	learn: 0.6582609	total: 9.06s	remaining: 42.7s
175:	learn: 0.6579568	total: 9.11s	remaining: 42.7s
176:	learn: 0.6577383	total: 9.17s	remaining: 42.6s
177:	learn: 0.6573184	total: 9.22s	remaining: 42.6s
178:	learn: 0.6570819	total: 9.27s	remaining: 42.5s
179:	learn: 0.6569295	total: 9.32s	remaining: 42.5s
180:	learn: 0.6567749	total: 9.37s	remaining: 42.4s
181:	learn: 0.6565962	total: 9.42s	remaining: 42.3s
182:	learn: 0.65

324:	learn: 0.6308583	total: 17.3s	remaining: 35.9s
325:	learn: 0.6307620	total: 17.3s	remaining: 35.8s
326:	learn: 0.6306552	total: 17.4s	remaining: 35.8s
327:	learn: 0.6306029	total: 17.4s	remaining: 35.7s
328:	learn: 0.6304881	total: 17.5s	remaining: 35.6s
329:	learn: 0.6303229	total: 17.5s	remaining: 35.6s
330:	learn: 0.6302350	total: 17.6s	remaining: 35.5s
331:	learn: 0.6300924	total: 17.6s	remaining: 35.5s
332:	learn: 0.6298670	total: 17.7s	remaining: 35.4s
333:	learn: 0.6297116	total: 17.7s	remaining: 35.4s
334:	learn: 0.6296443	total: 17.8s	remaining: 35.3s
335:	learn: 0.6295424	total: 17.8s	remaining: 35.3s
336:	learn: 0.6294515	total: 17.9s	remaining: 35.2s
337:	learn: 0.6293795	total: 17.9s	remaining: 35.1s
338:	learn: 0.6292228	total: 18s	remaining: 35.1s
339:	learn: 0.6290649	total: 18s	remaining: 35s
340:	learn: 0.6289497	total: 18.1s	remaining: 35s
341:	learn: 0.6288505	total: 18.1s	remaining: 34.9s
342:	learn: 0.6287930	total: 18.2s	remaining: 34.8s
343:	learn: 0.628659

486:	learn: 0.6130485	total: 25.8s	remaining: 27.2s
487:	learn: 0.6129531	total: 25.9s	remaining: 27.2s
488:	learn: 0.6128308	total: 25.9s	remaining: 27.1s
489:	learn: 0.6127454	total: 26s	remaining: 27s
490:	learn: 0.6126908	total: 26s	remaining: 27s
491:	learn: 0.6126019	total: 26.1s	remaining: 26.9s
492:	learn: 0.6124757	total: 26.2s	remaining: 26.9s
493:	learn: 0.6123324	total: 26.2s	remaining: 26.8s
494:	learn: 0.6122859	total: 26.3s	remaining: 26.8s
495:	learn: 0.6121762	total: 26.3s	remaining: 26.7s
496:	learn: 0.6120867	total: 26.4s	remaining: 26.7s
497:	learn: 0.6119978	total: 26.4s	remaining: 26.6s
498:	learn: 0.6117717	total: 26.5s	remaining: 26.6s
499:	learn: 0.6117367	total: 26.5s	remaining: 26.5s
500:	learn: 0.6115723	total: 26.6s	remaining: 26.5s
501:	learn: 0.6114906	total: 26.6s	remaining: 26.4s
502:	learn: 0.6114064	total: 26.7s	remaining: 26.4s
503:	learn: 0.6113641	total: 26.7s	remaining: 26.3s
504:	learn: 0.6113222	total: 26.8s	remaining: 26.2s
505:	learn: 0.611237

649:	learn: 0.6004684	total: 34.3s	remaining: 18.5s
650:	learn: 0.6004220	total: 34.4s	remaining: 18.4s
651:	learn: 0.6003697	total: 34.4s	remaining: 18.4s
652:	learn: 0.6003233	total: 34.5s	remaining: 18.3s
653:	learn: 0.6002378	total: 34.5s	remaining: 18.3s
654:	learn: 0.6001458	total: 34.6s	remaining: 18.2s
655:	learn: 0.6000978	total: 34.6s	remaining: 18.2s
656:	learn: 0.6000585	total: 34.7s	remaining: 18.1s
657:	learn: 0.5999431	total: 34.7s	remaining: 18.1s
658:	learn: 0.5998986	total: 34.8s	remaining: 18s
659:	learn: 0.5997637	total: 34.8s	remaining: 18s
660:	learn: 0.5996872	total: 34.9s	remaining: 17.9s
661:	learn: 0.5995889	total: 35s	remaining: 17.8s
662:	learn: 0.5995682	total: 35s	remaining: 17.8s
663:	learn: 0.5995150	total: 35s	remaining: 17.7s
664:	learn: 0.5994453	total: 35.1s	remaining: 17.7s
665:	learn: 0.5993890	total: 35.1s	remaining: 17.6s
666:	learn: 0.5993024	total: 35.2s	remaining: 17.6s
667:	learn: 0.5992259	total: 35.2s	remaining: 17.5s
668:	learn: 0.5991731	

811:	learn: 0.5902804	total: 42.5s	remaining: 9.85s
812:	learn: 0.5902210	total: 42.6s	remaining: 9.79s
813:	learn: 0.5901805	total: 42.6s	remaining: 9.74s
814:	learn: 0.5901273	total: 42.7s	remaining: 9.69s
815:	learn: 0.5900592	total: 42.7s	remaining: 9.64s
816:	learn: 0.5900006	total: 42.8s	remaining: 9.59s
817:	learn: 0.5899252	total: 42.9s	remaining: 9.54s
818:	learn: 0.5898491	total: 42.9s	remaining: 9.48s
819:	learn: 0.5897694	total: 43s	remaining: 9.43s
820:	learn: 0.5897352	total: 43s	remaining: 9.38s
821:	learn: 0.5896947	total: 43.1s	remaining: 9.32s
822:	learn: 0.5896593	total: 43.1s	remaining: 9.27s
823:	learn: 0.5895960	total: 43.2s	remaining: 9.22s
824:	learn: 0.5895364	total: 43.2s	remaining: 9.16s
825:	learn: 0.5894803	total: 43.3s	remaining: 9.11s
826:	learn: 0.5894356	total: 43.3s	remaining: 9.06s
827:	learn: 0.5894166	total: 43.4s	remaining: 9.01s
828:	learn: 0.5893759	total: 43.4s	remaining: 8.95s
829:	learn: 0.5892926	total: 43.5s	remaining: 8.9s
830:	learn: 0.589

972:	learn: 0.5823652	total: 50.5s	remaining: 1.4s
973:	learn: 0.5823261	total: 50.5s	remaining: 1.35s
974:	learn: 0.5822612	total: 50.6s	remaining: 1.3s
975:	learn: 0.5822244	total: 50.6s	remaining: 1.24s
976:	learn: 0.5821832	total: 50.7s	remaining: 1.19s
977:	learn: 0.5821438	total: 50.7s	remaining: 1.14s
978:	learn: 0.5820965	total: 50.8s	remaining: 1.09s
979:	learn: 0.5820546	total: 50.8s	remaining: 1.04s
980:	learn: 0.5819976	total: 50.9s	remaining: 985ms
981:	learn: 0.5819552	total: 50.9s	remaining: 933ms
982:	learn: 0.5819280	total: 51s	remaining: 881ms
983:	learn: 0.5819000	total: 51s	remaining: 829ms
984:	learn: 0.5818775	total: 51s	remaining: 777ms
985:	learn: 0.5818297	total: 51.1s	remaining: 725ms
986:	learn: 0.5817844	total: 51.1s	remaining: 674ms
987:	learn: 0.5817326	total: 51.2s	remaining: 622ms
988:	learn: 0.5817106	total: 51.3s	remaining: 570ms
989:	learn: 0.5816628	total: 51.3s	remaining: 518ms
990:	learn: 0.5816421	total: 51.3s	remaining: 466ms
991:	learn: 0.581597

/home/ben/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:57:13] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


VotingClassifier(estimators=[('rf', RandomForestClassifier()),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7fc6b05bf100>),
                             ('lgb', LGBMClassifier()),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            mo

In [34]:
preds = model_2.predict(X_test)
f1 = f1_score(y_test, preds, average='micro')
print("F1 Score", f1)

F1 Score 0.7399562564751928


## Modelo 3

In [38]:
estimators_3 = []

estimators_3.append(("cat", CatBoostClassifier(silent=True, random_seed=123)))
estimators_3.append(("rf", RandomForestClassifier(class_weight='balanced', min_samples_leaf=4,
                       min_samples_split=5, n_estimators=70, random_state=123)))
estimators_3.append(("rf_2", RandomForestClassifier(class_weight='balanced', max_features='sqrt',
                       min_samples_leaf=4, min_samples_split=9, n_estimators=70, random_state=123)))
estimators_3.append(("lgbm", lgb.LGBMClassifier(force_col_wise=False, force_row_wise=False, learning_rate=0.5,
               max_depth=10, n_estimators=40, num_leaves=30, objective='multiclass', random_state=123)))
estimators_3.append(("xgb", xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=np.nan, monotone_constraints='()',
              n_estimators=30, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=123, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)))
estimators_3.append(("adboost", AdaBoostClassifier(n_estimators=10, random_state=123)))

In [40]:
model_3 = VotingClassifier(estimators=estimators_3, voting='soft')
model_3.fit(X_train, y_train)

/home/ben/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:06:25] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


VotingClassifier(estimators=[('cat',
                              <catboost.core.CatBoostClassifier object at 0x7fc6ac81d280>),
                             ('rf',
                              RandomForestClassifier(class_weight='balanced',
                                                     min_samples_leaf=4,
                                                     min_samples_split=5,
                                                     n_estimators=70,
                                                     random_state=123)),
                             ('rf_2',
                              RandomForestClassifier(class_weight='balanced',
                                                     max_features='sqrt',
                                                     min_samples_leaf=4,
                                                     min_samples_split=9,
                                                     n_estimators=70...
                                            max_depth=15, 

In [41]:
preds = model_3.predict(X_test)
f1 = f1_score(y_test, preds, average='micro')
print("F1 Score", f1)

F1 Score 0.7432945781052147


## Test modelo 3

In [42]:
test = pd.read_csv('../Test/test_set_geo_levels_sum_mult.csv')

test.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,...,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,geo_level_1_id_sum,geo_level_2_id_sum,geo_level_3_id_sum,geo_level_sum,geo_level_multiply
0,300051,17,596,11307,3,20,7,6,0,1,...,0,0,0,0,1,7452,45,4,11920,114562524
1,99355,6,141,11987,2,25,13,5,0,1,...,0,0,0,0,1,8104,65,3,12134,10141002
2,890251,22,19,10044,2,5,4,5,0,1,...,0,0,0,0,1,2106,17,4,10085,4198392
3,745817,26,39,633,1,0,19,3,0,0,...,0,0,0,0,2,7387,1329,190,698,641862
4,421793,17,289,7970,3,15,8,7,0,1,...,0,0,0,0,1,7452,62,7,8276,39156610


In [43]:
ids = test['building_id']

test.drop('building_id', axis='columns', inplace=True)

test.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,geo_level_1_id_sum,geo_level_2_id_sum,geo_level_3_id_sum,geo_level_sum,geo_level_multiply
0,17,596,11307,3,20,7,6,0,1,0,...,0,0,0,0,1,7452,45,4,11920,114562524
1,6,141,11987,2,25,13,5,0,1,0,...,0,0,0,0,1,8104,65,3,12134,10141002
2,22,19,10044,2,5,4,5,0,1,0,...,0,0,0,0,1,2106,17,4,10085,4198392
3,26,39,633,1,0,19,3,0,0,0,...,0,0,0,0,2,7387,1329,190,698,641862
4,17,289,7970,3,15,8,7,0,1,0,...,0,0,0,0,1,7452,62,7,8276,39156610


In [44]:
preds = model_3.predict(test)
test['damage_grade'] = preds
test['building_id'] = ids

result = test[['building_id', 'damage_grade']]

result.head()

,building_id,damage_grade
0,300051,2
1,99355,2
2,890251,2
3,745817,1
4,421793,2


In [45]:
result.to_csv('../preds_ensamble_v1.csv', index=False)